In [9]:
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
import twitter_credentials
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="realDonaldTrump", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(10))


                                              tweets                   id  \
0  RT @DarrellIssa: Tonight, Joe Biden made it cl...  1311173279315615745   
1  RT @GaryCoby: “Joe Biden will smile in your fa...  1311173043193077763   
2  RT @TimRunsHisMouth: If you still can't figure...  1311172100766531584   
3  RT @KLF: BIDEN tonight: “The Green New Deal is...  1311171805533569024   
4  RT @RealSaavedra: Biden has repeatedly failed ...  1311171675292086272   
5  RT @BrandonStraka: @realDonaldTrump You cleane...  1311171071593320451   
6  RT @philipttirino: The American people didn’t ...  1311169630401179648   
7  RT @abigailmarone: It's literal insanity watch...  1311169552269692928   
8  RT @TrumpWarRoom: Tonight, the American people...  1311168608786149379   
9  RT @mikehahn_: Chris Wallace owes @realDonaldT...  1311168128982884353   

   len                date              source  likes  retweets  sentiment  
0  139 2020-09-30 05:17:24  Twitter for iPhone      0      4403          1 

In [1]:
import csv
import tweepy
import ssl
import twitter_credentials


consumer_key = twitter_credentials.CONSUMER_KEY 
consumer_secret = twitter_credentials.CONSUMER_SECRET
access_token = twitter_credentials.ACCESS_TOKEN
access_token_secret = twitter_credentials.ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
ssl._create_default_https_context = ssl._create_unverified_context
api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True)

user = api.me()
print (user.name)

Usha Chari


In [18]:
name = 'realDonaldTrump'
#tweet_id = '1311153253472636928'
tweet_id = '1311492120826449920'

In [19]:
replies=[]
for tweet in tweepy.Cursor(api.search,q='to:'+name, result_type='recent', timeout=999999).items(1000):
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==tweet_id):
            replies.append(tweet)

with open('twitter_trump_data.csv', 'a+') as f:
    csv_writer = csv.DictWriter(f, fieldnames=('user', 'text'))
    csv_writer.writeheader()
    for tweet in replies:
        row = {'user': tweet.user.screen_name, 'text': tweet.text.replace('\n', ' ')}
        csv_writer.writerow(row)

In [23]:
replies

[Status(_api=<tweepy.api.API object at 0x7f84c14d8fd0>, _json={'created_at': 'Thu Oct 01 03:19:40 +0000 2020', 'id': 1311506037187375106, 'id_str': '1311506037187375106', 'text': '@realDonaldTrump You were so afraid 😱 of him you wouldn’t let him speak!', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1311492120826449920, 'in_reply_to_status_id_str': '1311492120826449920', 'in_reply_to_user_id': 25073877, 'in_reply_to_user_id_str': '25073877', 'in_reply_to_screen_name': 'realDonaldTrump', 'user': {'id': 27513829, 'id_str': '27513829', 'name': 'Mango', 'screen_name': 'mango239', 'location': 'Victoria, British Columbia', 'description': ''

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

csv_file = "twitter_trump_data.csv"
election_data_df = pd.read_csv(csv_file)
election_data_df.head()

,user,text
0,carson101011,@realDonaldTrump https://t.co/jbiK0xaGBf
1,pedrojosecama,@realDonaldTrump https://t.co/7rkYCMY6G3
2,mrangelbabyGOD,@realDonaldTrump TRUMP WEARING HIS WHITE SUPR...
3,user,text
4,user,text


In [25]:
len(election_data_df)

66